In [1]:
import re
import collections
import itertools
from collections import Counter
from collections import OrderedDict

In [2]:
ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
A_LENGTH = 26
englFreqList = {'E': 12.70, 'T': 9.06, 'A': 8.17, 'O': 7.51, 'I': 6.97, 'N': 6.75, 'S': 6.33, 'H': 6.09, 'R': 5.99, 'D': 4.25, 'L': 4.03, 'C': 2.78, 'U': 2.76, 'M': 2.41, 'W': 2.36, 'F': 2.23, 'G': 2.02, 'Y': 1.97, 'P': 1.93, 'B': 1.29, 'V': 0.98, 'K': 0.77, 'J': 0.15, 'X': 0.15, 'Q': 0.10, 'Z': 0.07}
ENGLFREQ = 'ETAOINSHRDLCUMWFGYPBVKJXQZ'
LET_IN_SUBKEY = 4
MAX_KEY = 20 # can vary

In [22]:
ENGLISH_WORDS = loadDictionary()

In [4]:
def getFileText():
    with open('km2text.txt', 'r',encoding='utf-8') as f:
        text = f.read()
    return text

In [5]:
def countLetters(text):
    counts = Counter(text)
    return counts

In [6]:
def findNGramsAndDistances(message):
    pattern = re.compile('[^A-Z]')
    message = pattern.sub('', message.upper())
    seqSpacings = {} 
    for seqLen in range(3, 6):
        for seqStart in range(len(message) - seqLen):
            seq = message[seqStart:seqStart + seqLen]

            for i in range(seqStart + seqLen, len(message) - seqLen):
                if message[i:i + seqLen] == seq:
                    if seq not in seqSpacings:
                        seqSpacings[seq] = [] 

                    seqSpacings[seq].append(i - seqStart)
    return seqSpacings

In [7]:
def findFactors(number):
    arr = []
    for value in range(2, number+1):
        if(number % value == 0):
            arr.append(value)
    return arr

In [8]:
def findListOfListFactors(arr):
    factorsArr = []
    for val in arr.values():
        for item in val:
                factorsArr.extend(findFactors(item))

    return factorsArr

In [9]:
def listOfAllMostCommonFactors(arr):
    arr2 = collections.Counter(arr)
    return arr2.most_common()

In [10]:
def getPossibleKeys2(arr):
    allFact = findListOfListFactors(arr)
    mostCommonList = listOfAllMostCommonFactors(allFact)
    arr2 = [item[0] for item in mostCommonList if item[0]<= MAX_KEY]
    
    return arr2

In [11]:
def getStringsFromEveryNthLetter(msg,start,length):
    pattern = re.compile('[^A-Z]')
    msg = pattern.sub('', msg)
    return msg[start::length]

In [12]:
def  mostCommonFactor(arr):
    return arr[0][1]

In [13]:
def freqTable(msg,keyLength):
    table = []
    for i in range(keyLength):
        nthLettersString = getStringsFromEveryNthLetter(msg,i,keyLength)
        freq = []
        for k in ALPHABET:
            decrString = decryptMessage(k,nthLettersString)
            freqTuples = (k,freqOfMostCommonLet(decrString))
            freq.append(freqTuples)
        
        sortedFreq = sorted(freq, key=lambda tup: tup[1],reverse = True)
        
        table.append(sortedFreq[:LET_IN_SUBKEY])
        
    return table

In [14]:
def decryptMessage(key,msg):
    res = [] 
    keyIndex = 0
    key = key.upper()
    for s in msg: 
        num = ALPHABET.find(s.upper())
        if num == -1:
            res.append(s)
        else:
            num -= ALPHABET.find(key[keyIndex])%A_LENGTH
            
            if s.isupper():
                res.append(ALPHABET[num])
            if s.islower():
                res.append(ALPHABET[num].lower())
                
            keyIndex += 1 
            if keyIndex == len(key):
                keyIndex = 0

    return ''.join(res)

In [15]:
def getStringOfFreq(msg):
    counts = Counter(msg)
    dictOfFreq = {}
    
    for l in ALPHABET:
        if counts[l] not in dictOfFreq:
            dictOfFreq[counts[l]] = [l]
        else:
            dictOfFreq[counts[l]].append(l)
            
    for f in dictOfFreq:
        dictOfFreq[f].sort(key=ENGLFREQ.find, reverse = True)
    
    sortedDict = OrderedDict(sorted(dictOfFreq.items(), key=lambda t: t[0], reverse = True))
    sortedStr = ''.join(symb for val in sortedDict.values() for symb in val)
        
    return sortedStr

In [16]:
def freqOfMostCommonLet(msg):
    sortedStr = getStringOfFreq(msg)
    matches1 = set(sortedStr[:6]) & set(ENGLFREQ[:6])
    matches2 = set(sortedStr[-6:]) & set(ENGLFREQ[-6:])
    
    return len(matches1) + len(matches2)

In [17]:
def freqTableToCombList(table,keyLength):
    length = len(table)
    arr = [0]*length
    
    for l in range(length):
        arr[l] = [i for val in table[l] for i in val[0]]
        
    
    return itertools.product(*arr)

In [26]:
def tryToHack(msg,keyLength):
    upperMsg = msg.upper()
    table = freqTable(upperMsg,keyLength)
    keysComb = freqTableToCombList(table,keyLength)
    
    for i in keysComb:
        keyStr = ''.join(s for val in i for s in val)
        #print('Ключ: ', keyStr)
        decrMsg = decryptMessage(keyStr, upperMsg)
        
        if isEnglish(decrMsg):
            print('Похоже ли это на ответ?')
            print(decrMsg[:100], '.....\n')
            option = check_user_input('Введите 1,чтобы завершить,2,чтобы продолжить \n')
            if option==1:
                print('Ключ: ', keyStr)
                return backTextToOrigin(msg,decrMsg)
            else:
                continue

            
    return False

In [19]:
def backTextToOrigin(msg,decrMsg):
    origMsg = []
    for i in range(len(msg)):
        if msg[i].isupper():
            origMsg.append(decrMsg[i].upper())
        else:
            origMsg.append(decrMsg[i].lower())
    
    return ''.join(origMsg)

In [20]:
UPPERLETTERS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
LETTERS_AND_SPACE = UPPERLETTERS + UPPERLETTERS.lower() + ' \t\n'

def loadDictionary():
    dictionaryFile = open('abc.txt')
    englishWords = {}
    for word in dictionaryFile.read().split('\n'):
        englishWords[word] = None
    dictionaryFile.close()
    return englishWords


def getEnglishCount(message):
    message = message.upper()
    message = removeNonLetters(message)
    possibleWords = message.split()

    if possibleWords == []:
        return 0.0

    matches = 0
    for word in possibleWords:
        if word in ENGLISH_WORDS:
            matches += 1
    return float(matches) / len(possibleWords)


def removeNonLetters(message):
    lettersOnly = []
    for symbol in message:
        if symbol in LETTERS_AND_SPACE:
            lettersOnly.append(symbol)
    return ''.join(lettersOnly)


def isEnglish(message, wordPercentage=30, letterPercentage=85):
    wordsMatch = getEnglishCount(message) * 100 >= wordPercentage
    numLetters = len(removeNonLetters(message))
    messageLettersPercentage = float(numLetters) / len(message) * 100
    lettersMatch = messageLettersPercentage >= letterPercentage
    return wordsMatch and lettersMatch

In [21]:
def check_user_input(prompt):
    while True:
        try:
            opt = int(input(prompt))
        except ValueError:
            print("Ошибка ввода")
            continue

        if opt not in [1,2]:
            continue
        else:
            break
    return opt

In [23]:
msg = getFileText()
print('Зашифрованное сообщение:\n ',msg[:100])
step1 = findNGramsAndDistances(msg)

Зашифрованное сообщение:
  TCE EOI AKO GZVEY

Mu. axh Prn. Uurnlhy, yj quhser aoxr, Zvlvzk Drdvh,
wovh pmfud oo vai xkao khet w


In [28]:
step2 = getPossibleKeys2(step1)
step2

[2, 4, 3, 6, 12, 8, 9, 16, 5, 10, 15, 7, 14, 11, 13]

In [29]:
print("Попробуем подобрать ключ...")
for k in step2:
    decrMsg = tryToHack(msg,k)
    if(decrMsg):
        break

    
print('Исходный текст: ',decrMsg)    

Попробуем подобрать ключ...
Похоже ли это на ответ?
THE BOY WHO LIVED

MR. AND MRS. DURSLEY, OF NUMBER FOUR, PRIVET DRIVE,
WERE PROUD TO SAY THAT THEY W .....

Введите 1,чтобы завершить,2,чтобы продолжить 
1
Ключ:  AVADAKEDAVRA
Исходный текст:  THE BOY WHO LIVED

Mr. and Mrs. Dursley, of number four, Privet Drive,
were proud to say that they were perfectly normal,
thank you very much. They were the last people you’d
expect to be involved in anything strange or
mysterious, because they just didn’t hold with such
nonsense.

Mr. Dursley was the director of a firm called
Grunnings, which made drills. He was a big, beefy
man with hardly any neck, although he did have a
very large mustache. Mrs. Dursley was thin and
blonde and had nearly twice the usual amount of
neck, which came in very useful as she spent so
much of her time craning over garden fences, spying
on the neighbors. The Dursley s had a small son
called Dudley and in their opinion there was no finer
boy anywhere.

The Dursleys ha